In [1]:
import numpy as np
from tensorflow import keras

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
from classifier_net import ClassifierNet

test_classifier = ClassifierNet(input_shape=input_shape, num_classes=num_classes)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
test_classifier.train(x_train, y_train)

Train on 54000 samples, validate on 6000 samples
54000/54000 [==============================] - 1s 22us/sample - loss: 0.4514 - acc: 0.8775 - val_loss: 0.1985 - val_acc: 0.9482


In [4]:
score = test_classifier.evaluate(x_test, y_test)

print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.2352085318863392
Test accuracy: 0.9333
